In [3]:
import sys
import json
from collections import Counter
sys.path.append('../../../')

from helper import FunctionName as fn

umbra_contract_txs_json = "../../../umbra/data/polygon/umbra_contract_txs.json"
skr_contract_txs_json = "../../../umbra/data/polygon/stealth_key_registry_contract_txs.json"

with open(umbra_contract_txs_json, "r") as file:
    data = json.load(file)

with open(skr_contract_txs_json, "r") as file:
    skr_data = json.load(file)

contract_txs = data["result"]
skr_contract_txs = skr_data["result"]

In [4]:
"""
    ### heuristics_4 --> ###
    #1
    - How many sponsors are there?
    - What's going on with the sponsor fee? Are they the same in some cases?
    - Maybe something with the relayers?
    - Is there any connection between the sponsor and priority fee? Are there equal sponsor and priority fees?
"""

sponsors = set()
fees = []
relayers = []

for d in contract_txs:
    if d["functionName"] == fn.W_TOKEN.value:
        sponsors.add(d[d["functionName"]]["_sponsor"])
        fees.append(d[d["functionName"]]["_sponsorFee"])
        relayers.append(d["from"])


print(sponsors)
print(list(filter(lambda d: d[1] > 1, dict(Counter(fees)).items())))
print(dict(Counter(relayers)))
print()

sponsorFee_list = []
maxPriorityFeePerGas_list = []

for d in contract_txs:
    if "maxPriorityFeePerGas" in d:
        # it means it is a tx with type EIP-1559

        if d["functionName"] == fn.W_TOKEN.value:
            sponsorFee_list.append(d[d["functionName"]]["_sponsorFee"])
            continue

        maxPriorityFeePerGas_list.append(d["maxPriorityFeePerGas"])


        if d["functionName"] == fn.S_ETH.value:
            receiver = d[d["functionName"]]["_receiver"]
            
            for tx in d[d["functionName"]][receiver]:
                if "maxPriorityFeePerGas" in tx:
                    maxPriorityFeePerGas_list.append(tx["maxPriorityFeePerGas"])


for d in skr_contract_txs:
    if "maxPriorityFeePerGas" in d:
        # it means it is a tx with type EIP-1559

        maxPriorityFeePerGas_list.append(d["maxPriorityFeePerGas"])

for f in sponsorFee_list:
    if f in maxPriorityFeePerGas_list:
        print(f)

"""
    - So there's only one sponsor for the tokens,
    - And the fees are always different (except 1), so there's no need to check for the same values for possible
    connections.
    - There are two relayers, maybe it has changed once during the operation of the umbra. We can't really use this for
    anything.
    - Since there is no result for same sponsor fee and priority fee, it is clear now that there is no connection
    between sponsor and priority fees.

    Based on the results, we won't deal with token txs in this heuristics.
"""
pass

{'0xb4435399AB53D6136C9AEEBb77a0120620b117F9'}
[(10000000000000000, 4), (100000000000000000, 404), (100000, 1528), (500000, 241), (500000000000000000, 19), (250000, 456), (250000000000000000, 38), (143437500000000000, 2), (189375000000000000, 3), (188437, 25), (189375, 8), (190312, 3), (185625, 3), (183750, 2), (180000, 7), (180937, 6), (181875, 2), (182812, 5), (180000000000000000, 3), (187500, 5), (188437500000000000, 2), (187500000000000000, 3), (184687, 3), (186562, 5), (177187, 3), (174375000000000000, 4), (171562, 19), (171562500000000000, 6), (174375, 8), (173437, 9), (170625, 4), (172500, 16), (173437500000000000, 2), (175312, 3), (176250, 3), (167812500000000000, 3), (161250000000000000, 3), (161250, 24), (160312, 34), (162187500000000000, 4), (162187, 8), (155625, 7), (159375, 31), (158437, 13), (157500, 15), (156562, 12), (150937500000000000, 5), (151875, 11), (152812, 5), (150937, 14), (155625000000000000, 2), (169687, 13), (169687500000000000, 4), (165000, 4), (165937, 7),

In [5]:
"""
    Get the fees used by an address

    Originally this is not that useful since even if the sender and the withdrawal is the same person, the "from" address
    of the sender is different from the stealth. So it makes more sense to use the "to" address of the withdrawal txs.
"""

result_dict = {}

for d in contract_txs:    
    if "maxFeePerGas" in d:
        # It means it is a tx with type EIP-1559

        if d["functionName"] == fn.W_TOKEN.value:
            continue

        if d["from"] not in result_dict:
            result_dict[d["from"]] = []

        temp_append = {
            "maxFeePerGas": d["maxFeePerGas"], 
            "maxPriorityFeePerGas": d["maxPriorityFeePerGas"]
        }

        if d["functionName"] == fn.S_ETH.value:
            temp_append["type"] = "s_eth"
        # we also inspect the token txs, because it can make our measurments more precise
        elif d["functionName"] == fn.S_TOKEN.value:
            temp_append["type"] = "s_token"

        result_dict[d["from"]].append(temp_append)

        if d["functionName"] == fn.S_ETH.value:
            receiver = d[d["functionName"]]["_receiver"]

            for tx in d[d["functionName"]][receiver]:
                if "maxFeePerGas" in tx:
                    if tx["to"] not in result_dict:
                        result_dict[tx["to"]] = []

                    temp_append = {
                        "maxFeePerGas": tx["maxFeePerGas"], 
                        "maxPriorityFeePerGas": tx["maxPriorityFeePerGas"],
                        "type": "w_eth"
                    }

                    result_dict[tx["to"]].append(temp_append)

for d in skr_contract_txs:
    if "maxFeePerGas" in d:
        # it means it is a tx with type EIP-1559

        if d["from"] not in result_dict:
            result_dict[d["from"]] = []

        temp_append = {
            "maxFeePerGas": d["maxFeePerGas"], 
            "maxPriorityFeePerGas": d["maxPriorityFeePerGas"],
            "type": "registry"
        }

        result_dict[d["from"]].append(temp_append)

with open("../results/fees_by_addresses.json", "w") as file:
    json.dump(result_dict, file)

"""
    Currently there is not much information that we could get out from this measurement, so we won't use it yet.
"""
pass

In [ ]:
# TODO make good figures